In [ ]:
import openai
import json
import os

In [ ]:
# Define the functions
def getproductavailabilitydatebyzip(zip, productcode, deliveryservicedays):
    # Add your implementation here
    pass

def getproductinventorybyloc(loc):
    # Add your implementation here
    pass

def getproductprice(zip, customerid, productcode):
    # Add your implementation here
    pass

In [ ]:
# Map the function names to the functions
available_functions = {
    "getproductavailabilitydatebyzip": getproductavailabilitydatebyzip,
    "getproductinventorybyloc": getproductinventorybyloc,
    "getproductprice": getproductprice,
}

In [ ]:
def run_conversation():
    # Define the system message
    system_message = "You are a helpful Customer Service chatbot answering questions about products, especially delivery times, product availability, and product price."

    # Define the function descriptions for the API
    functions = [
        {
            "name": "getproductavailabilitydatebyzip",
            "description": "Returns a delivery for a product and zip",
            "parameters": {"zip": "string", "productcode": "string", "deliveryservicedays": "int"},
        },
        {
            "name": "getproductinventorybyloc",
            "description": "Returns the loc id, inventory and open orders for the loc if provided, otherwise provide a list of locs",
            "parameters": {"loc": "string"},
        },
        {
            "name": "getproductprice",
            "description": "Returns the price of an item, refined by zip and or customer id if provided",
            "parameters": {"zip": "string", "customerid": "string", "productcode": "string"},
        },
    ]

    # Start the conversation with the user's question
    user_question = input("Ask your question: ")
    messages = [{"role": "system", "content": system_message}, {"role": "user", "content": user_question}]

    # Call the model
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto",
    )
    response_message = response["choices"][0]["message"]

    # Check if GPT wanted to call a function
    if response_message.get("function_call"):
        function_name = response_message["function_call"]["name"]
        fuction_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = fuction_to_call(**function_args)

        # Send the info on the function call and function response to GPT
        messages.append(response_message)
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
        )
        return second_response

In [ ]:

# Run the conversation
print(run_conversation())